In [685]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [686]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [687]:
train_data = pd.read_csv('spaceship-titanic/train.csv')
test_data = pd.read_csv('spaceship-titanic/test.csv')
sample_submission = pd.read_csv('spaceship-titanic/sample_submission.csv')
sample_submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [688]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [689]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [690]:
train_data["HomePlanet"] = train_data["HomePlanet"].fillna(train_data["HomePlanet"].mode()[0])
train_data["CryoSleep"] = train_data["CryoSleep"].fillna(False)
train_data["Cabin"] = train_data["Cabin"].fillna(train_data["Cabin"].mode()[0])
train_data["Destination"] = train_data["Destination"].fillna(train_data["Destination"].mode()[0])
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean())
train_data["VIP"] = train_data["VIP"].fillna(False)
train_data["RoomService"] = train_data["RoomService"].fillna(train_data["RoomService"].mean())
train_data["FoodCourt"] = train_data["FoodCourt"].fillna(train_data["FoodCourt"].mean())
train_data["ShoppingMall"] = train_data["ShoppingMall"].fillna(train_data["ShoppingMall"].mean())
train_data["Spa"] = train_data["Spa"].fillna(train_data["Spa"].mean())
train_data["VRDeck"] = train_data["VRDeck"].fillna(train_data["VRDeck"].mean())

test_data["HomePlanet"] = test_data["HomePlanet"].fillna(test_data["HomePlanet"].mode()[0])
test_data["CryoSleep"] = test_data["CryoSleep"].fillna(False)
test_data["Cabin"] = test_data["Cabin"].fillna(test_data["Cabin"].mode()[0])
test_data["Destination"] = test_data["Destination"].fillna(test_data["Destination"].mode()[0])
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].mean())
test_data["VIP"] = test_data["VIP"].fillna(False)
test_data["RoomService"] = test_data["RoomService"].fillna(test_data["RoomService"].mean())
test_data["FoodCourt"] = test_data["FoodCourt"].fillna(test_data["FoodCourt"].mean())
test_data["ShoppingMall"] = test_data["ShoppingMall"].fillna(test_data["ShoppingMall"].mean())
test_data["Spa"] = test_data["Spa"].fillna(test_data["Spa"].mean())
test_data["VRDeck"] = test_data["VRDeck"].fillna(test_data["VRDeck"].mean())

In [691]:
LabelEncoder = LabelEncoder()

train_data["HomePlanet"] = LabelEncoder.fit_transform(train_data["HomePlanet"])
train_data["CryoSleep"] = LabelEncoder.fit_transform(train_data["CryoSleep"])
train_data["Cabin"] = LabelEncoder.fit_transform(train_data["Cabin"])
train_data["Destination"] = LabelEncoder.fit_transform(train_data["Destination"])
train_data["VIP"] = LabelEncoder.fit_transform(train_data["VIP"])
train_data["Transported"] = LabelEncoder.fit_transform(train_data["Transported"])

test_data["HomePlanet"] = LabelEncoder.fit_transform(test_data["HomePlanet"])
test_data["CryoSleep"] = LabelEncoder.fit_transform(test_data["CryoSleep"])
test_data["Cabin"] = LabelEncoder.fit_transform(test_data["Cabin"])
test_data["Destination"] = LabelEncoder.fit_transform(test_data["Destination"])
test_data["VIP"] = LabelEncoder.fit_transform(test_data["VIP"])

In [692]:
train_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1,0,149,2,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,0,0,2184,2,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,1,0,1,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,1,0,1,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,0,0,2186,2,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,1,0,146,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0
8689,9278_01,0,1,5280,1,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0
8690,9279_01,0,0,5285,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1
8691,9280_01,1,0,2131,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0


In [693]:
# データの準備
X_train = train_data.drop(["PassengerId", "Name", "Transported"], axis=1)
y_train = train_data["Transported"]

X_test = test_data.drop(["PassengerId", "Name"], axis=1)

# 線形回帰モデル
model = LinearRegression()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 予測結果を0または1に変換（閾値0.5）
y_pred_bool = (y_pred >= 0.5).astype(bool)

# 予測値を表示
print(y_pred_bool)

[ True False  True ...  True  True  True]


In [694]:
predict_data = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Transported": y_pred_bool})

predict_data.to_csv("predict.csv", index=False)